In [ ]:
import pandas as pd

In [2]:
# Lendo o dataframe dos emprestimos

df_emprestimos = pd.read_csv('../data/interim/emprestimos-concat.csv')

# Mostrando alguns emprestimos
df_emprestimos.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2.008023e+09,ALUNO DE GRADUAÇÃO
1,709685,L167050,NaN,2010/01/04 07:44:10.750000000,2010/01/12 07:34:13.934000000,2.008023e+09,ALUNO DE GRADUAÇÃO
2,709686,2006017618,2010/01/26 08:07:01.738000000,2010/01/04 08:08:44.081000000,2010/02/25 07:36:25.800000000,2.008112e+09,ALUNO DE PÓS-GRADUAÇÃO
3,709687,L184117,2010/01/18 11:07:46.470000000,2010/01/04 08:24:21.284000000,2010/02/03 08:58:45.692000000,2.007211e+08,ALUNO DE GRADUAÇÃO
4,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2.008023e+09,ALUNO DE GRADUAÇÃO


In [3]:
df_emprestimos.columns

Index(['id_emprestimo', 'codigo_barras', 'data_renovacao', 'data_emprestimo',
       'data_devolucao', 'matricula_ou_siape', 'tipo_vinculo_usuario'],
      dtype='object')

In [4]:
# Lendo o dataframe com os emprestimos
df_exemplares = pd.read_parquet('../data/raw/dados_exemplares.parquet')

# Mostrando alguns exemplares
df_exemplares.head()

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
0,5,L000003,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,694,1
1,4,L000002,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,688,1
2,3,L000001,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,638,1
3,7,L000114,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,616,5
5,10,L000041,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,657,15


In [8]:
# Renomeando as colunas codigo_barras para referenciar aos respectivos dataframes

df_emprestimos.rename(columns={'codigo_barras':'codigo_barras_emprestimo'}, inplace=True)

df_exemplares.rename(columns={'codigo_barras':'codigo_barras_exemplar'}, inplace=True)

In [9]:
df_emprestimos.columns

Index(['id_emprestimo', 'codigo_barras_emprestimo', 'data_renovacao',
       'data_emprestimo', 'data_devolucao', 'matricula_ou_siape',
       'tipo_vinculo_usuario'],
      dtype='object')

In [10]:
df_exemplares.columns

Index(['id_exemplar', 'codigo_barras_exemplar', 'colecao', 'biblioteca',
       'status_material', 'localizacao', 'registro_sistema'],
      dtype='object')

In [11]:
# Unindo os dataframes

df = pd.merge(df_emprestimos, df_exemplares, left_on='codigo_barras_emprestimo', right_on='codigo_barras_exemplar')

In [ ]:
# Mostrando alguns registros do dataframe unido
df.head()

In [13]:
# Retirando colunas inúteis (registro_sistema)
df.drop(columns=['registro_sistema'], inplace=True)

In [14]:
df.columns

Index(['id_emprestimo', 'codigo_barras_emprestimo', 'data_renovacao',
       'data_emprestimo', 'data_devolucao', 'matricula_ou_siape',
       'tipo_vinculo_usuario', 'id_exemplar', 'codigo_barras_exemplar',
       'colecao', 'biblioteca', 'status_material', 'localizacao'],
      dtype='object')

Os itens do acervo em uma biblioteca são organizados por um sistema de classificação de acordo com o respectivo tema. Existem diversos sistemas, mas este conjunto está de acordo com a CDU - Classificação Decimal Universal. Esta classificação é decimal, pois varia de acordo com a classe de cada assunto:


000 a 099: Generalidades. Ciência e conhecimento.

100 a 199: Filosofia e psicologia.

200 a 299: Religião.

300 a 399: Ciências sociais.

400 a 499: Classe vaga. Provisoriamente não ocupada.

500 a 599: Matemática e ciências naturais.

600 a 699: Ciências aplicadas.

700 a 799: Belas artes.

800 a 899: Linguagem. Língua. Linguística.

900 a 999: Geografia. Biografia. História."

In [15]:
# Classificando o arcevo da biblioteca com .cut()
# o tamanho de rotulos deve ser igual a len(caixas)-1

caixas = [000, 99, 199, 299, 399, 499, 599, 699, 799, 899, 999]

rotulos = [
    'Generalidades. Ciência e conhecimento',
    'Filosofia e psicologia',
    'Religião',
    'Ciências sociais',
    'Classe vaga. Provisoriamente não ocupada',
    'Matemática e ciências naturais',
    'Ciências aplicadas',
    'Belas artes',
    'Linguagem. Língua. Linguística',
    'Geografia. Biografia. História.',
]

df['CDU'] = pd.cut(df['localizacao'], bins=caixas, labels=rotulos, right=True, include_lowest=True)

In [16]:
# A coluna da matricula (“matricula_ou_siape”) não está com um formato muito legível. Transforme-a em formato String.
# O desafio pediu para transformar a coluna em string, mas o pandas assimila uma string como object

df = df.assign(matricula_ou_siape=df['matricula_ou_siape'].astype('object'))

df.dtypes

id_emprestimo                  int64
codigo_barras_emprestimo      object
data_renovacao                object
data_emprestimo               object
data_devolucao                object
matricula_ou_siape            object
tipo_vinculo_usuario          object
id_exemplar                    int64
codigo_barras_exemplar        object
colecao                       object
biblioteca                    object
status_material               object
localizacao                    int64
CDU                         category
dtype: object

In [17]:
df.to_csv('../data/processed/emprestimos_exemplares.csv')